In [1]:
import pandas as pd
import numpy as np
import os

os.chdir('/home/kruu/git_folder/eye_tracking/')
from utils.data_processing import EyeTrackingProcessor

import torch
from torch.utils.data import DataLoader
from models.lstm_classifier import LSTMClassifier, train_classifier, evaluate_model, split_by_participant
from utils.dataset import GazeMouseDataset

****
# Import Datasets
****

In [2]:
data_path = "/store/kruu/eye_tracking"
files_list = os.listdir(data_path)
files_list = [os.path.join(data_path, file) for file in files_list]

tasks = ['Task 1', 'Task 2', 'Task 3', 'Task 4', 'Task 5', 'Task 6']
features = ['Recording timestamp', 'Gaze point X', 'Gaze point Y', 'Mouse position X', 'Mouse position Y', 'Event', 'Participant name']
interpolate_col = ['Recording timestamp', 'Gaze point X', 'Gaze point Y', 'Mouse position X', 'Mouse position Y', 'Blink']

processor = EyeTrackingProcessor()
all_data = processor.load_data(files_list)
dataset = processor.get_features(all_data, tasks, features)
dataset, blinks = processor.detect_blinks(dataset)

# Fixed Time step resampling
dataset_time_resampled = processor.resample_tasks_fixed_time(dataset, interpolate_col, timestep = 0.01)
dataset_time_resampled.Blink = (dataset_time_resampled.Blink > 0.5) #Transform interpolated data
# dataset_time_resampled = processor.pad_tasks(dataset_time_resampled) # No need to pad here
dataset_time_resampled["id"] = dataset_time_resampled["Participant name"].astype(str) + "_" + dataset_time_resampled["Task_id"].astype(str) + "_" + dataset_time_resampled["Task_execution"].astype(str)

/home/kruu/git_folder/eye_tracking/utils/data_processing.py:16: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep='\t')
/home/kruu/git_folder/eye_tracking/utils/data_processing.py:16: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep='\t')


In [3]:
dataset_time_resampled["Mouse position X"] = dataset_time_resampled["Mouse position X"].ffill().bfill()
dataset_time_resampled["Mouse position Y"] = dataset_time_resampled["Mouse position Y"].ffill().bfill()
dataset_time_resampled["Gaze point X"] = dataset_time_resampled["Gaze point X"].ffill().bfill()
dataset_time_resampled["Gaze point Y"] = dataset_time_resampled["Gaze point Y"].ffill().bfill()

****
# Classifier
****

In [4]:
features = ['Relative timestamp', 'Gaze point X', 'Gaze point Y', 'Mouse position X', 'Mouse position Y', 'Blink']

train_df, val_df, test_df = split_by_participant(dataset_time_resampled, val_split=0.2, test_split=0.1)

model, mean, std = train_classifier(train_df,
                                    val_df,
                                    features,
                                    batch_size=32,
                                    hidden_dim=64,
                                    num_layers=2,
                                    learning_rate=0.001,
                                    num_epochs=100)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: kruu (zhaw_zav) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/home/kruu/.conda/envs/aware/lib/python3.12/site-packages/pytorch_lightning/core/optimizer.py:378: Found unsupported keys in the optimizer configuration: {'gradient_clip_val'}

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | lstm      | LSTM             | 5.1 K  | train
1 | fc        | Linear           | 198    | train
2 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
5.3 K     Trainable params
0         Non-trainable params
5.3 K     Total params
0.021     Total estimated model params size (MB)
3         Modules in train mode
0         Modules in eval mode
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/kruu/.conda/envs/aware/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=51` in the `DataLoader` to improve performance.


/home/kruu/.conda/envs/aware/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=51` in the `DataLoader` to improve performance.
/home/kruu/.conda/envs/aware/lib/python3.12/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (9) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 99: 100%|██████████| 9/9 [00:28<00:00,  0.31it/s, v_num=0pxd, train_loss_step=1.290, val_loss=1.870, train_loss_epoch=1.380]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 9/9 [00:28<00:00,  0.31it/s, v_num=0pxd, train_loss_step=1.290, val_loss=1.870, train_loss_epoch=1.380]


In [5]:
print(mean, std)

Relative timestamp    1.034442e+07
Gaze point X          7.386896e+02
Gaze point Y          5.382008e+02
Mouse position X      7.072564e+02
Mouse position Y      4.283481e+02
Blink                 5.888771e-01
dtype: float64 Relative timestamp    8.288185e+06
Gaze point X          5.536564e+02
Gaze point Y          3.946933e+02
Mouse position X      5.675037e+02
Mouse position Y      3.430667e+02
Blink                 4.920380e-01
dtype: float64


****
# Analysis
****

In [4]:
ckpt_path = "/home/kruu/git_folder/eye_tracking/GazeMouse_Classification/2ajznad2/checkpoints/best_lstm_classifier.ckpt"
features = ['Relative timestamp', 'Gaze point X', 'Gaze point Y', 'Mouse position X', 'Mouse position Y']
    
model_load = LSTMClassifier(input_dim=5, hidden_dim=64, num_classes = 6, num_layers=4, learning_rate=0.001)
model_load.load_state_dict(torch.load(ckpt_path)["state_dict"])
model_load.eval()

LSTMClassifier(
  (lstm): LSTM(5, 64, num_layers=4, batch_first=True)
  (fc): Linear(in_features=64, out_features=6, bias=True)
  (criterion): CrossEntropyLoss()
)

In [ ]:
res = evaluate_model(model_load, test_df, mean, std, features)

✅ Evaluation Complete: Loss = 3.0381, Accuracy = 0.1852
